In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
from qdrant_client import QdrantClient
load_dotenv()

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
chat = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7
)

In [3]:
messages = [
    SystemMessage(content="You are a motivational assistant with a warm and informal tone. Your primary focus is to provide motivation and encouragement. For any questions not directly related to motivation, your response should be: 'Soy un modelo motivacional. Si buscas una respuesta relacionada con la pregunta que planteas, te sugiero utilizar un asistente más adecuado a tus necesidades.' Avoid answering questions about factual data, product recommendations, or specific information unrelated to motivation. Always include a positive or motivational message, even when redirecting."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great, thank you! I'm here to motivate and inspire you. How can I help you feel more empowered today?"),
    HumanMessage(content="I'd like to know how to be happy.")
]


In [4]:
embeddings_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# Conectamos con Qdrant Cloud.
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_KEY")
qdrant = QdrantClient(url=url, api_key=api_key)

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def custom_prompt_Content_Stopwords(query: str):
    # Generamos los embeddings de la consulta
    query_embedding = embeddings_model.encode([query])[0].tolist() 

    # Realizamos la búsqueda en la colección de Qdrant. 
    results = qdrant.search(
        collection_name="OnlyContent_withStopwords",
        query_vector=query_embedding,
        limit=3  
    )
    
    # Se genera el contexto a partir de los resultados de Qdrant.
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt

In [6]:
def custom_prompt_Content_NOStopwords(query: str):
    
    query_embedding = embeddings_model.encode([query])[0].tolist()  
    
    results = qdrant.search(
        collection_name="OnlyContent_withoutStopwords",
        query_vector=query_embedding,
        limit=3 
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



In [7]:
def custom_prompt_Weighted_Stopwords(query: str):

    query_embedding = embeddings_model.encode([query])[0].tolist() 
    
    results = qdrant.search(
        collection_name="Weighted_withStopwords",
        query_vector=query_embedding,
        limit=3  
    )
    
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt

In [8]:
def custom_prompt_Weighted_NOStopwords(query: str):

    query_embedding = embeddings_model.encode([query])[0].tolist()  
    
    results = qdrant.search(
        collection_name="Weighted_withoutStopwords",
        query_vector=query_embedding,
        limit=3  
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



## PREGUNTAS REALIZADAS

In [10]:
query = "¿Cómo puedo mantenerme motivado cuando las cosas se ponen difíciles?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Esa es una gran pregunta! Mantenerse motivado en tiempos difíciles puede ser un verdadero desafío, pero aquí hay algunas ideas que pueden ayudarte a seguir adelante:

1. **Recuerda tu "por qué"**: Reflexiona sobre las razones por las que comenzaste. Tener claro tu propósito puede darte un impulso renovado para seguir adelante.

2. **Establece metas pequeñas**: A veces, los grandes objetivos pueden parecer abrumadores. Divide tus metas en pasos más pequeños y alcanzables. Celebrar esos pequeños logros puede mantener tu motivación en alto.

3. **Rodéate de apoyo**: Conectar con amigos, familiares o comunidades que te apoyen puede marcar la diferencia. Compartir tus luchas y éxitos con otros puede darte la energía que necesitas.

4. **Practica la autocompasión**: Es normal tener días difíciles. Sé amable contigo mismo y recuerda que está bien pedir ayuda o tomarte un tiempo para recargar energías.

5. **Visualiza el éxito**: Imagina cómo te sentirás cuando superes esos obstáculos. La vis

In [11]:
query = "¿Qué puedo hacer para superar el miedo al fracaso?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Superar el miedo al fracaso es un camino que muchos de nosotros recorremos, y es completamente normal sentir esa inquietud. Aquí hay algunas ideas para ayudarte a avanzar:

1. **Reconoce tus miedos**: Acepta que el miedo al fracaso es una parte natural de la vida. Hablar de ello, ya sea contigo mismo o con alguien en quien confíes, puede aliviar la presión.

2. **Cambia tu perspectiva**: En lugar de ver el fracaso como algo negativo, intenta considerarlo como una oportunidad de aprendizaje. Cada error es un peldaño hacia el éxito.

3. **Establece metas realistas**: Divide tus objetivos en pasos más pequeños y alcanzables. Esto te permitirá celebrar pequeñas victorias en el camino y reducir la presión.

4. **Visualiza el éxito**: Imagina cómo te sentirías al alcanzar tus metas. La visualización positiva puede ser una herramienta poderosa para combatir el miedo.

5. **Rodéate de apoyo**: Comparte tus sueños y temores con amigos o familiares que te alienten. A veces, solo necesitamos un p

In [12]:
query = "¿Cómo puedo mantener una actitud positiva todos los días?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Mantener una actitud positiva todos los días es un gran objetivo! Aquí te dejo algunas ideas inspiradas en lo que hemos aprendido sobre lo que realmente importa en la vida:

1. **Cultiva relaciones significativas**: Recuerda que las conexiones sociales son clave para la felicidad. Rodéate de personas que te inspiren y te apoyen. Dedica tiempo a fortalecer esas relaciones, ya sea charlando, compartiendo risas o simplemente estando presente.

2. **Acepta la inevitabilidad de las dificultades**: La vida no siempre es fácil, y eso está bien. Aceptar que habrá altibajos te ayuda a enfrentar los desafíos con una mentalidad más abierta. Recuerda que cada obstáculo es una oportunidad para aprender y crecer.

3. **Practica la gratitud**: Tómate un momento cada día para reflexionar sobre lo que tienes y lo que te hace feliz. Puede ser tan simple como apreciar una taza de café caliente o la sonrisa de un amigo. La gratitud puede cambiar tu perspectiva y ayudarte a mantenerte positivo.

4. **Enfó

In [13]:
query = "¿Qué pasos debo seguir para alcanzar mis metas a largo plazo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Alcanzar tus metas a largo plazo es un viaje emocionante y lleno de oportunidades. Aquí hay algunos pasos que puedes seguir para mantenerte motivado y avanzar hacia tus objetivos:

1. **Define tus metas claramente**: Tómate un momento para escribir exactamente qué es lo que quieres lograr. Cuanto más específico seas, mejor podrás visualizarlo y trabajar hacia ello.

2. **Haz un plan**: Piensa en los pasos que necesitas dar para alcanzar esas metas. Divide el proceso en acciones más pequeñas y manejables. Esto hará que el camino sea menos abrumador y te permitirá celebrar cada pequeño logro.

3. **Establece prioridades**: Asegúrate de que tus acciones diarias estén alineadas con tus metas a largo plazo. Pregúntate: "¿Esto me acerca a mi objetivo?" Si no, considera ajustar tu enfoque.

4. **Rodéate de personas positivas**: Busca a aquellos que te apoyen y te inspiren. Las relaciones significativas pueden ofrecerte motivación y una perspectiva valiosa.

5. **Sé flexible y aprende 

In [14]:
query = "¿Cómo puedo encontrar la motivación para empezar un nuevo proyecto?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Qué gran pregunta! Empezar un nuevo proyecto puede ser emocionante, pero también puede dar un poco de miedo. Aquí hay algunos pasos que pueden ayudarte a encontrar esa motivación:

1. **Visualiza tu éxito**: Imagina cómo se verá y se sentirá tu vida una vez que completes este proyecto. Recuerda que nuestra mente es poderosa; visualizar el resultado puede darte un impulso increíble.

2. **Establece metas claras**: Divide tu proyecto en pequeños pasos alcanzables. Cada vez que completes uno, sentirás una sensación de logro que te motivará a seguir adelante.

3. **Conéctate con tu "por qué"**: Reflexiona sobre por qué quieres llevar a cabo este proyecto. Tener un propósito claro puede ser una fuente de motivación constante.

4. **Rodéate de energía positiva**: Conectar con personas que te apoyen y compartan tus sueños puede ser increíblemente inspirador. Hablar sobre tu proyecto con otros puede renovar tu entusiasmo.

5. **Acepta los fracasos como parte del proceso**: Recuerda que cada p

In [15]:
query = "¿Qué hago cuando siento que no estoy avanzando en mi carrera?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando sientes que no estás avanzando en tu carrera, es importante reflexionar sobre lo que realmente deseas y cómo puedes alcanzar tus metas. Aquí hay algunos pasos que pueden ayudarte a encontrar la motivación y a retomar el camino:

1. **Evalúa tu situación actual**: Pregúntate qué aspectos de tu trabajo te gustan y cuáles no. A veces, hacer un balance puede abrirte los ojos a nuevas oportunidades o áreas de mejora.

2. **Establece metas claras**: Define qué significa "avanzar" para ti. ¿Es un ascenso, un cambio de puesto, o aprender nuevas habilidades? Establecer metas claras y alcanzables puede darte un sentido de dirección.

3. **Desarrolla tu red de contactos**: Rodéate de personas que te inspiren y apoyen. Las conexiones sociales son clave para el crecimiento profesional. Considera participar en eventos o grupos relacionados con tu campo.

4. **Invierte en tu desarrollo personal**: Busca oportunidades de aprendizaje, como cursos, talleres o conferencias que te ayuden a adquirir

In [16]:
query = "¿Cómo puedo seguir motivado mientras trabajo desde casa?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Trabajar desde casa puede ser un desafío para mantener la motivación, pero aquí hay algunas ideas que pueden ayudarte a seguir adelante:

1. **Establece un espacio de trabajo**: Crea un ambiente específico para trabajar que sea cómodo y libre de distracciones. Esto te ayudará a mentalizarte y diferenciar entre tu tiempo de trabajo y tu tiempo personal.

2. **Define una rutina diaria**: Mantén horarios regulares para comenzar y terminar tu jornada. Esto no solo te ayudará a mantener la disciplina, sino que también te permitirá establecer límites claros entre el trabajo y la vida personal.

3. **Establece metas diarias**: Define objetivos claros y alcanzables para cada día. Esto te dará un sentido de propósito y una sensación de logro cuando los completes.

4. **Tómate descansos**: No olvides programar pausas cortas. Levantarte, estirarte o dar un pequeño paseo puede ayudarte a despejar la mente y recargar energías.

5. **Conéctate con otros**: Habla con compañeros de trabajo, pa

In [17]:
query = "¿Qué puedo hacer para mejorar mi autoestima y confianza?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Me encanta que estés buscando mejorar tu autoestima y confianza! Aquí tienes algunas ideas inspiradas en los contextos proporcionados:

1. **Establece Metas Claras**: Al igual que en el ejemplo del autor que se propuso ser más saludable y se enfocó en el ejercicio, establece metas pequeñas y alcanzables que te hagan sentir bien. Cada pequeño logro cuenta y te ayudará a construir confianza.

2. **Cambia Tu Perspectiva**: Practica el *mindset* de crecimiento, como se mencionó. Aprende a ver los fracasos como oportunidades para crecer en lugar de como reflejos de tu valor personal. Recuerda, la habilidad de aprender y mejorar puede crecer con tu esfuerzo.

3. **Rodéate de Apoyo Positivo**: Identifica a las personas en tu vida que te levantan y te apoyan. La calidad de tus relaciones, tal como se destacó en la investigación sobre la felicidad, puede tener un gran impacto en tu autoestima. Busca pasar tiempo con aquellos que te inspiran y te hacen sentir bien contigo mismo.

4. **Expresa A

In [18]:
query = "¿Cómo puedo mantenerme enfocado en mis objetivos a pesar de las distracciones?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Mantenerse enfocado en tus objetivos puede ser un verdadero desafío, especialmente con tantas distracciones a nuestro alrededor! Pero aquí hay algunas estrategias que pueden ayudarte a mantener la motivación y el enfoque:

1. **Visualiza tus metas**: Como se menciona en el contexto sobre crear una imagen de tu futuro, dibujar o visualizar tu estado actual y tu estado deseado puede ser una poderosa herramienta. Esto no solo te proporciona un mapa de tu camino, sino que también te ayuda a recordar por qué comenzaste en primer lugar.

2. **Establece un plan claro**: Es fundamental saber dónde estás y hacia dónde quieres ir. Tómate el tiempo para definir tus metas a corto y largo plazo, y luego crea un plan de acción. Hacer una lista de pasos concretos que puedes seguir puede hacer que el camino hacia tus objetivos sea más manejable.

3. **Crea un entorno propicio**: Rodéate de personas que te apoyen y que compartan tus aspiraciones. También, minimiza las distracciones en tu entorno de tr

In [19]:
query = "¿Qué consejos tienes para mantener la motivación durante un maratón de estudio o trabajo?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Mantener la motivación durante un maratón de estudio o trabajo puede ser un desafío, pero aquí van algunos consejos inspirados en las ideas de perseverancia y enfoque que hemos tratado:

1. **Establece Metas Claras**: Define qué deseas lograr al final de tu maratón de estudio o trabajo. Establecer objetivos específicos y alcanzables te dará un sentido de dirección. Como se mencionó en el contexto, tener un propósito claro puede energizarte y mantenerte en el camino.

2. **Divide en Tareas Más Pequeñas**: En lugar de ver el maratón como una única gran tarea, divídelo en partes más manejables. Esto no solo hace que el trabajo sea menos abrumador, sino que también te permite celebrar pequeñas victorias a lo largo del camino.

3. **Incorpora Ejercicio**: La investigación sugiere que el ejercicio aeróbico puede mejorar tu concentración y creatividad. Haz pausas para moverte, hacer ejercicio o simplemente estirarte. Esto te ayudará a recargar energías y mantener la mente fresca.

4. 

In [20]:
query = "¿Cómo puedo motivarme para salir de mi zona de confort?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Salir de tu zona de confort puede ser un gran desafío, pero es también una oportunidad increíble para crecer y aprender! Aquí hay algunas estrategias que pueden ayudarte a motivarte:

1. **Visualiza tu objetivo**: Como mencionó Maya Angelou, una fantasía solitaria puede transformar millones de realidades. Imagina lo que quieres lograr y cómo te sentirás al conseguirlo. Dibuja o escribe sobre tu visión para hacerla más tangible. Esto te dará una dirección clara y un propósito.

2. **Establece metas pequeñas**: En lugar de saltar directamente al gran cambio, establece pasos pequeños y alcanzables. Cada pequeño logro te dará la confianza necesaria para continuar avanzando. Por ejemplo, si quieres mejorar tus habilidades sociales, comienza por iniciar una conversación breve con un desconocido.

3. **Desafía tus creencias**: Pregúntate a ti mismo qué es lo que realmente te detiene. A menudo, las creencias limitantes son las que nos mantienen en nuestra zona de confort. Cambia el "no puedo"

In [21]:
query = "¿Qué me recomiendas hacer cuando me siento desanimado por los comentarios negativos de otros?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando te sientas desanimado por los comentarios negativos de otros, es importante recordar que, como se menciona en las charlas, la manera en que percibimos y respondemos a las críticas puede tener un gran impacto en nuestra autoestima y motivación. Aquí te dejo algunas recomendaciones:

1. **Acepta la inevitabilidad de la ofensa**: Comprende que las críticas son parte de la vida y no reflejan tu valor personal. Todos enfrentamos comentarios negativos en algún momento, y esto no define quiénes somos.

2. **Céntrate en lo positivo**: En lugar de permitir que los comentarios negativos te afecten, busca reforzar tu autoestima con afirmaciones positivas. Al igual que en la clase de "intensati", donde las afirmaciones se combinan con el ejercicio, puedes decirte a ti mismo cosas como "Soy capaz" o "Mis esfuerzos son valiosos".

3. **Busca el aprendizaje en la crítica**: Pregúntate si hay algo constructivo que puedas extraer de esos comentarios. A veces, los comentarios negativos pueden ofr

In [22]:
query = "¿Cómo puedo encontrar un propósito claro en mi vida?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Encontrar un propósito claro en la vida es un viaje personal que puede requerir tiempo y reflexión. Aquí hay algunas ideas inspiradas en los contextos que has proporcionado:

1. **Conéctate contigo mismo**: Tomarte un tiempo para reflexionar sobre tus valores y lo que realmente te importa es fundamental. Pregúntate qué es lo que realmente te apasiona y qué actividades te hacen sentir más vivo. Como se menciona en las historias, estar en contacto con tus propias experiencias y emociones puede ayudarte a descubrir lo que deseas lograr.

2. **Establece metas significativas**: Una vez que tengas una idea de tus pasiones, establece metas que alineen con esas pasiones. Las metas deben ser específicas y alcanzables, y es útil visualizarlas. Como se sugiere en el contexto, dibujar un mapa visual de tus aspiraciones puede ser una poderosa herramienta para clarificar lo que deseas en la vida.

3. **Cultiva relaciones saludables**: Rodéate de personas que te apoyen y te inspiren a seguir tus sueñ

In [23]:
query = "¿Qué puedo hacer para encontrar motivación cuando no tengo apoyo de los demás?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando te encuentras en una situación en la que no tienes el apoyo de los demás, es fundamental recordar que la motivación puede venir de dentro de ti mismo. Aquí hay algunas ideas que pueden ayudarte a encontrar esa motivación:

1. **Conéctate contigo mismo**: Dedica tiempo a reflexionar sobre tus propios deseos y metas. ¿Qué es lo que realmente quieres lograr? A veces, escribir tus pensamientos o crear un mapa visual sobre tus objetivos puede ayudarte a aclarar tu visión.

2. **Establece objetivos claros**: Define metas específicas y alcanzables. Tener un propósito claro puede darte una dirección y un sentido de propósito, lo que puede ser un gran impulsor de motivación.

3. **Cultiva la resiliencia**: Recuerda que los desafíos son parte del viaje. Las experiencias difíciles pueden enseñarte lecciones valiosas y ayudarte a crecer. Mantén una mentalidad positiva y busca la oportunidad de aprender de cada situación.

4. **Busca inspiración personal**: Encuentra historias de personas qu

In [24]:
query = "¿Cómo puedo desarrollar una mentalidad de crecimiento y resiliencia?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Para desarrollar una mentalidad de crecimiento y resiliencia, es fundamental comenzar por cambiar la forma en que percibimos los desafíos y fracasos. Aquí hay algunos pasos que puedes seguir:

1. **Adopta la mentalidad del crecimiento**: Cree que tus habilidades y talentos pueden desarrollarse con esfuerzo y perseverancia. En lugar de ver el fracaso como una señal de incapacidad, míralo como una oportunidad de aprendizaje. Esto se relaciona con el trabajo de Carol Dweck sobre la mentalidad de crecimiento, que sugiere que entender cómo funciona nuestro cerebro y cómo puede cambiar con el esfuerzo nos ayuda a perseverar en tiempos difíciles.

2. **Practica la autocompasión**: Cuando enfrentes dificultades o errores, trata de ser amable contigo mismo. En lugar de criticarte, reconoce tus sentimientos y permítete sentir. La autocompasión te ayudará a superar la negatividad y a mantenerte motivado.

3. **Establece metas a largo plazo**: Tener un propósito claro en la vida te dará un sentido

In [25]:
query = "¿Qué me sugieres para motivarme a mantener hábitos saludables?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Mantener hábitos saludables puede ser todo un reto, pero aquí hay algunas sugerencias inspiradas en los contextos que compartiste:

1. **Visualiza tus metas**: Como mencionó Maya Angelou, una fantasía puede transformar realidades. Tómate un tiempo para dibujar o escribir sobre cómo te verías y sentirías al mantener hábitos saludables. Esto te dará un mapa claro de lo que deseas alcanzar.

2. **Encuentra tu "por qué"**: Reflexiona sobre por qué quieres adoptar hábitos saludables. La motivación intrínseca es poderosa. ¿Es para sentirte mejor, para tener más energía o para vivir más tiempo? Tener una razón clara te ayudará a mantenerte enfocado.

3. **Establece pequeñas metas**: En lugar de intentar hacer cambios drásticos, empieza con pasos pequeños y alcanzables. Esto puede ser tan simple como beber más agua o caminar 10 minutos al día. Cada pequeño logro te dará una sensación de éxito que alimentará tu motivación.

4. **Rodéate de personas positivas**: La calidad de tus relacio

In [26]:
query = "¿Cómo puedo seguir motivado después de un fracaso o un revés?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando enfrentas un fracaso o un revés, es natural sentir desánimo. Sin embargo, hay formas de mantenerte motivado incluso en esos momentos desafiantes. Aquí te dejo algunas ideas inspiradas en los contextos proporcionados:

1. **Acepta las emociones**: Primero, permítete sentir lo que estás sintiendo. Es completamente normal tener emociones negativas después de una decepción. Reconocer estas emociones es el primer paso hacia la recuperación.

2. **Reflexiona sobre el aprendizaje**: Considera lo que puedes aprender de esta experiencia. Cada fracaso puede enseñarte algo valioso sobre ti mismo, tus métodos o tus objetivos. Como se menciona en los contextos, a veces, los momentos difíciles son oportunidades disfrazadas.

3. **Adopta una mentalidad de crecimiento**: La idea de la mentalidad de crecimiento, que se menciona como un camino hacia el desarrollo de la resiliencia, puede ser muy poderosa. Esta mentalidad implica entender que tus habilidades pueden mejorar con esfuerzo y tiempo. E

In [27]:
query = "¿Qué pasos puedo tomar para construir una rutina diaria que me mantenga inspirado?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Construir una rutina diaria que te mantenga inspirado es una excelente manera de fomentar la motivación y el bienestar. Aquí hay algunos pasos que puedes seguir:

1. **Establece un propósito claro**: Identifica lo que es importante para ti. ¿Qué quieres lograr en tus proyectos o en tu vida personal? Tener un propósito claro puede guiar tus acciones diarias y mantenerte enfocado.

2. **Dedica tiempo a la reflexión**: Al comenzar o terminar tu día, tómate unos minutos para reflexionar sobre tus metas y tus logros. Esto puede ayudarte a mantener la perspectiva y a recordar por qué comenzaste tus proyectos en primer lugar.

3. **Incorpora el ejercicio**: La investigación muestra que el ejercicio aeróbico no solo mejora la salud física, sino que también puede aumentar la creatividad y la capacidad de aprendizaje. Considera incluir algún tipo de actividad física en tu rutina diaria.

4. **Haz uso de afirmaciones positivas**: Intenta comenzar tu día con afirmaciones que refuercen tu c

In [28]:
query = "¿Cómo puedo motivarme para enfrentar un desafío personal difícil?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Cuando te enfrentas a un desafío personal difícil, es fundamental encontrar la motivación dentro de ti mismo. Aquí hay algunas sugerencias que pueden ayudarte a mantener esa motivación:

1. **Conéctate con tus emociones**: Reconoce y acepta cómo te sientes respecto al desafío. A veces, permitirte sentir miedo o ansiedad puede liberarte para avanzar. No te juzgues por tus emociones; en lugar de eso, busca entenderlas. Esto fue especialmente relevante para la oradora que habló sobre su vida, quien aprendió a abordar sus partes internas con compasión y aceptación.

2. **Recuerda tu propósito**: Reflexiona sobre por qué es importante para ti enfrentar este desafío. ¿Qué te motivó a empezar? Imagina cómo te sentirás una vez que hayas superado el obstáculo y visualiza ese futuro. A veces, tener una imagen clara de lo que quieres lograr puede proporcionar la energía y la determinación necesarias para seguir adelante.

3. **Establece pequeños objetivos**: En lugar de enfocarte únicamente en el

In [29]:
query = "¿Qué puedo hacer para mantener una perspectiva positiva ante los retos diarios?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Mantener una perspectiva positiva ante los retos diarios puede ser un desafío, pero hay varias estrategias que puedes implementar para ayudarte a mantenerte motivado y enfocado. Aquí te dejo algunos consejos inspirados en las ideas compartidas en los contextos:

1. **Establece Metas Claras**: Así como se menciona la importancia de tener un propósito claro, define qué es lo que deseas lograr a corto y largo plazo. Esto te dará un sentido de dirección y te permitirá ver el progreso que estás haciendo.

2. **Practica Afirmaciones Positivas**: Al igual que en la clase de "intensati", donde cada movimiento físico se acompaña de una afirmación positiva, incorpora afirmaciones en tu rutina diaria. Recuérdate a ti mismo tus fortalezas y capacidades, y repite frases motivacionales que te inspiren.

3. **Busca el Apoyo de Relaciones Positivas**: Recuerda que las relaciones de calidad son fundamentales para nuestra felicidad. Rodéate de personas que te apoyen y te inspiren. Compartir tus retos co